In [5]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import math

import pymorphy2
import re
import sqlite3

from collections import defaultdict
import time
import pickle
#MS SQL 
import pyodbc

from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [6]:
#MS SQL
server = 'NUTS' 
database = 'MK_news' 
username = '' 
password = '' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [7]:
cmd = "select doc_text from Documents"
cursor.execute(cmd)

In [9]:
#тестовая выборка для Московского комсомольца 
test_doc_to_clast = defaultdict(list)
analyzer = pymorphy2.MorphAnalyzer()  
categories = ['sport','economics','culture'] 
doc_id = 0
d=[]
for n_clast,  c in enumerate(categories):
    
    cmd = "select doc_text from Documents where category = ?"
    cursor.execute(cmd,c)
    
    for t in cursor.fetchall()[:100]:      
        test_doc_to_clast[n_clast].append(doc_id)
        article = t[0]
        d.append(str(parse_text(article)))
        #print( n_clast, c, doc_id,article,"\n")
        doc_id+=1
        #print(doc_id)
x = np.array(d) 
len(x)

300

In [8]:
stopWords = stopwords.words('russian')
stopWords.extend(['что', 'это', 'так', 'вот','ещё', 'свой' , 'быть','й', 'как', 'в', '—', 'к', 'на','свой', 'который', 'е','кстати', 'также'])

def parse_text(text):
    #print(type(text))
    words = (word for word in re.split('\W+', text) if (len(word) >0 )&( analyzer.normal_forms(word)[0].lower() not in stopWords ))
    lexems = (analyzer.normal_forms(word)[0] for word in words)
    
    return list(lexems)

In [10]:
x = np.array(d) 
type(x[1])

numpy.str_

In [11]:
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

X = count_vect.fit_transform(x)
X = tfidf_transformer.fit_transform(X)

In [12]:
k_means = KMeans(n_clusters = 3)

In [13]:
k_means.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [16]:
labels = k_means.labels_

In [22]:
labels[:100]

array([0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0,
       2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2,
       2, 2, 2, 0, 0, 2, 0, 2])

In [36]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs


In [45]:
plt.rcParams['figure.figsize'] = (16, 9)

# Creating a sample dataset with 4 clusters
X, y = make_blobs(n_samples=800, n_features=3, centers=4)

In [46]:
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X[:, 0], X[:, 1], X[:,2])

In [47]:
C = k_means.cluster_centers_

In [48]:
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=y)
ax.scatter(C[:, 0], C[:, 1], C[:, 2], marker='*', c='#050505', s=1000)